## Importing the necessary packages

In [ ]:
import cv2
import numpy as np
import glob
from tqdm import tqdm
import sys
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Loading and preprocessing images from the DeepGlobe dataset

In [ ]:
# Download dataset if not already present.
!curl -L -o deepglobe-land-cover-classification-dataset.zip https://www.kaggle.com/api/v1/datasets/download/balraj98/deepglobe-land-cover-classification-dataset
!unzip deepglobe-land-cover-classification-dataset.zip -d deepglobe-land-cover-classification-dataset

In [ ]:
path = os.path.join("deepglobe-land-cover-classification-dataset")

img_shape = (2448, 2448, 3) # Original image shape
input_shape = (512, 512, 3) # Model image shape
out_shape = (512, 512, 1) # Output mask shape

In [ ]:
def preprocess_img(img, shape, is_mask = False):
    # Images are converted from BGR to RGB.
    # Then, their resolution is lowered to accomodate our memory constraints.

    image = cv2.resize(cv2.imread(img), shape, interpolation = cv2.INTER_LINEAR if not is_mask else cv2.INTER_NEAREST)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def preprocess_mask(img, shape):
    # Transforming masks from 3 channel to a singular class channel with values from 0 to 6.
    mask = np.array(np.round(preprocess_img(img, shape, is_mask = True) / 255.0), dtype = np.uint8)
    mask_refined = 4 * mask[:,:,0] + 2 * mask[:,:,1] + mask[:,:,2]
    mask_refined = mask_refined * (mask_refined != 7) + 4 * (mask_refined == 7)
    return mask_refined

def get_dataset(mask_dirs):
    # Each mask file is matched to its corresponding satellite image.
    sat_dirs = [d.replace("_mask.png", "_sat.jpg") for d in mask_dirs]

    X = tf.convert_to_tensor([preprocess_img(file, input_shape[:2]) for file in sat_dirs], dtype=tf.uint8)
    y = tf.convert_to_tensor([preprocess_mask(file, out_shape[:2]) for file in mask_dirs], dtype=tf.uint8)

    return X, y

In [ ]:
# Loading the labeled data.
train_path = os.path.join(path, "train")
mask_dirs = glob.glob(os.path.join(train_path,"*_mask.png"))

n = len(mask_dirs)
train_split = 0.9
n_train = int(n * train_split)
n_val = n - n_train

X_train, y_train = get_dataset(mask_dirs[:n_train])
print(f"Loaded {n_train} training images.")

#X_val, y_val = get_dataset(os.path.join(path, "valid"))
#X_test, y_test = get_dataset(os.path.join(path, "test"))

In [ ]:
X_val, y_val = get_dataset(mask_dirs[n_train:n])
print(f"Loaded {n_val} validation images.")

## Model definition

In [ ]:
# 2 consecutive padded convolutions with ReLU activations as outlined in the U-Net paper.
# Is the basis for both in the Encoder and the Decoder.

class ConvBlock(tf.keras.Layer):
    def __init__(self, n_filters, filter_size):
        super().__init__()
        self.cn1 = tfl.Conv2D(n_filters, filter_size,
                            padding = "same", use_bias = True)
        self.relu1 = tfl.ReLU()
        self.bn1 = tfl.BatchNormalization()
        self.cn2 = tfl.Conv2D(n_filters, filter_size,
                            padding = "same", use_bias = True)
        self.relu2 = tfl.ReLU()
        self.bn2 = tfl.BatchNormalization()

    def call(self, x):
        x = self.relu1(self.bn1(self.cn1(x)))
        x = self.relu2(self.bn2(self.cn2(x)))
        return x

In [ ]:
class UNetConv(ConvBlock):
    def __init__(self, n_filters, filter_size, pool_stride):
        super().__init__(n_filters, filter_size)
        # Pooling layer to reduce image resolution.
        self.pool = tfl.MaxPooling2D(pool_stride)

    def call(self, x):
        x = super().call(x)
        return x, self.pool(x)


class UNetTConv(ConvBlock):
    def __init__(self, filters, filter_size, tfilters, tfilter_size):
        super().__init__(filters, filter_size)
        # ConvTranspose to restore image resolution, should us the same filter size as the pool stride.
        self.dcn = tfl.Conv2DTranspose(tfilters, tfilter_size, tfilter_size, padding = "same")

    def call(self, x):
        return self.dcn(super().call(x))

In [ ]:
n_blocks = 4 # Number of encoder and decoder block.
log_conv_filters = 5 # Number of filters in the first convolutional layer is 2^log_conv_filters.
# Increased by a factor of 2 for each subsequent layer. Inverse order for the decoder.
n_classes = 7 # Number of possible classes for each output pixel.
filter_size = 3 # Convolution filter size: 3*3
tfilter_size = 2 # Transposed convolution filter size: 2*2, should revert the pooling operation.
dropout = 0.3

In [ ]:
class UNET(tf.keras.Model):

    def __init__(self, blocks):
        super().__init__()
        self.scale = tfl.Rescaling(1.0 / 255.0)
        self.convs = [UNetConv(int(2**(log_conv_filters+i)), filter_size, 2) for i in range(blocks)] # encoder
        self.dropouts = [tfl.Dropout(dropout) for _ in range(blocks)] # Dropout is used at each decoder step.
        self.tconvs = [UNetTConv(int(2**(log_conv_filters + blocks - i)), filter_size,\
                int(2**(log_conv_filters + blocks -i)), tfilter_size) for i in range(blocks)] # decoder

        self.convf = ConvBlock(int(2**(log_conv_filters)), filter_size) # Final convolution before classification.
        self.conv1x1 =  tfl.Conv2D(n_classes, 1, padding = "same") # Convolution across the channels to get 1 probability per class.
        # self.softmax = tfl.Softmax(axis = -1)

    def call(self, x, training = False):
        x = self.scale(x)
        xs = []
        for conv in self.convs:
            # storing the unpooled feature maps for skip connections.
            skip, x = conv(x)
            xs.append(skip)

        skip = None
        for tconv, dropout in zip(self.tconvs, self.dropouts):
            x = x if skip is None else tf.concat([skip, x], axis = -1)
            x = tconv(dropout(x, training = training))
            skip = xs.pop()

        x = self.convf(tf.concat([skip, x], axis = -1))
        x = self.conv1x1(x)

        return x

In [ ]:
inputs = tf.keras.Input(input_shape)
model = UNET(n_blocks)
model(inputs)
model.summary()

## Training

In [ ]:
epsilon = 1e-9 # To avoid log(0).

freqs = np.zeros((n_classes,))
for c in range(n_classes):
    freqs[c] = np.mean(y_train.numpy() == c)

unknown_class = 0
beta = 0.1
class_weights = (1 - beta) / (1 - beta**freqs)
class_weights[unknown_class] = 0.0
class_weights = tf.constant(class_weights, dtype = tf.float32)

@tf.function
def SparseCategoricalCrossentropy(y_true, y_pred):
    # y_true = (batch_size, 1) -> one hot (batch_size, n_classes) -> frequency coefficient (each class would have its own weight instead of 1)
    y_true = tf.one_hot(tf.reshape(y_true,(-1,)), n_classes) * class_weights # removing the unknown class
     # (B, n_classes) * (n_classes,). Broadcasts.
    # y_pred = (batch_size, n_classes)
    y_pred = tf.reshape(y_pred, (-1, n_classes))
    y_pred = y_pred - tf.reduce_max(y_pred, axis = -1, keepdims = True)
    log_prob = - tf.math.log(tf.nn.softmax(y_pred, axis = -1) + epsilon)

    return tf.reduce_sum(y_true * log_prob) / tf.reduce_sum(tf.cast(y_true, tf.float32))

In [ ]:
@tf.function
def DiceLoss(y_true, y_pred):
    # Class weights?
    y_true = tf.one_hot(tf.reshape(y_true,(-1,)), n_classes)[..., 1:]
    y_pred = tf.reshape(y_pred, (-1, n_classes)) [..., 1:]
    y_pred = tf.nn.softmax(y_pred, axis = -1)

    return 1 - 2 * tf.reduce_sum(y_pred * y_true) / tf.reduce_sum(y_true  + y_pred)

In [ ]:
# F1 per class is used as the evaluation metric.
f1 = tf.keras.metrics.F1Score(average = "weighted")
@tf.function
def f1_score_images(y_true, y_pred):
    # Includes empty class.
    f1.reset_state()
    y_true_flat = tf.one_hot(tf.reshape(y_true,(-1,)), n_classes)
    y_pred_flat = tf.one_hot(tf.reshape(tf.argmax(y_pred, axis = -1), (-1,)), n_classes)
    return f1(y_true_flat, y_pred_flat)

In [ ]:
@tf.function
def IoU(y_true, y_pred):
    y_true = tf.one_hot(tf.reshape(y_true, (-1,)), n_classes)
    y_pred = tf.one_hot(tf.reshape(tf.argmax(y_pred, axis = -1), (-1,)), n_classes)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true + y_pred) - intersection
    return intersection / union

In [ ]:
lr = 3e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    lr,
    decay_steps=500,
    decay_rate=0.96,
    staircase=True)

#DiceLoss #tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, ignore_class = 0) #SparseCategoricalCrossentropy
loss_fn = DiceLoss
optimizer = tf.keras.optimizers.AdamW(lr_schedule)

In [ ]:
def augment_image(X, y, t = 0.2, r = 20.0):
    # Translation.
    dy = tf.random.uniform([], -t, t) * input_shape[0]
    dx = tf.random.uniform([], -t, t) * input_shape[1]

    translation_matrix = tf.convert_to_tensor([[1, 0, -dx],
                                            [0, 1, -dy],
                                            [0, 0, 1]], dtype=tf.float32)

    # Rotation.
    rotation_angle = (tf.random.uniform([], -r, r)) * np.pi / 180.0
    cos = tf.cos(rotation_angle)
    sin = tf.sin(rotation_angle)
    rotation_matrix = tf.convert_to_tensor([[cos, -sin, 0],
                                        [sin, cos, 0],
                                        [0, 0, 1]], dtype=tf.float32)

    # First 6 entries of 3x3 matrix are the affine parameters, 7th and 8th should just be 0.
    transform_matrix = tf.cast(rotation_matrix @ translation_matrix, tf.float32)
    transform_params = tf.reshape(transform_matrix, (1, 9))[:, :8]

    X = tf.keras.ops.image.affine_transform(
    X,
    transform_params,
    interpolation='nearest',
    fill_mode='nearest',
    data_format='channels_last'
    )

    y  = tf.keras.ops.image.affine_transform(
    tf.expand_dims(y, axis = -1),
    transform_params,
    interpolation='nearest',
    fill_mode='nearest',
    data_format='channels_last'
    )

    return X, tf.squeeze(y, axis = -1)

In [ ]:
t = 0.2 # translation factor
r = 0.0 # maximum absolute rotation angle

@tf.function
def train_step(X, y):
    X, y = augment_image(X, y, t, r)

    with tf.GradientTape() as tape:
        preds = model(X, training = True)
        loss = loss_fn(y, preds)
    f1_score = f1_score_images(y, preds)
    iou = IoU(y, preds)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss, f1_score, iou

In [ ]:
@tf.function
def val_step(X, y):
    preds = model(X, training = False)
    loss = loss_fn(y, preds)
    f1_score = f1_score_images(y, preds)
    iou = IoU(y, preds)
    return loss, f1_score, iou

In [ ]:
history = {}
batch_size = 8
EPOCHS = 50
best_ckpt = 1.0

# tqdm logs to stderr by default.
stderr = sys.stderr
sys.stderr = sys.stdout
checkpoint_path = './best.weights.h5'

In [ ]:
for epoch in range(EPOCHS):
    # Training loop
    train_loss, train_f1, train_iou = 0, 0, 0

    for i in tqdm(range(0, X_train.shape[0], batch_size)):
        X_batch = X_train[i: i + batch_size]
        y_batch = y_train[i: i + batch_size]
        loss, f1_score, iou = train_step(X_batch, y_batch)
        train_loss += loss * X_batch.shape[0]
        train_f1 += f1_score * X_batch.shape[0]
        train_iou += iou * X_batch.shape[0]

    train_loss /= n_train
    train_f1 /= n_train
    train_iou /= n_train

    history["loss"] = history.get("loss", []) + [train_loss]
    history["f1"] = history.get("f1", []) + [train_f1]

    val_loss, val_f1, val_iou = 0, 0, 0

    for j in range(0, X_val.shape[0], batch_size):
        X_batch_val = X_val[j: j + batch_size]
        y_batch_val = y_val[j: j + batch_size]
        loss, f1_score, iou = val_step(X_batch_val, y_batch_val)
        val_loss += loss * X_batch_val.shape[0]
        val_f1 += f1_score * X_batch_val.shape[0]
        val_iou += iou * X_batch_val.shape[0]

    val_loss /= n_val
    val_f1 /= n_val
    val_iou /= n_val

    if val_loss < best_ckpt:
        model.save_weights(checkpoint_path)
        best_ckpt = val_loss

    history["val_loss"] = history.get("val_loss", []) + [val_loss]
    history["val_f1"] = history.get("val_f1", []) + [val_f1]

    print(f"Epoch {epoch + 1} / {EPOCHS} loss: {train_loss:0.4f}, F1: {train_f1:0.4f}, IoU: {train_iou:0.4f} val_loss: {val_loss:0.4f}, val_f1: {val_f1:0.4f}, val IoU: {val_iou:0.4f}")

In [ ]:
# We reconfigure stderr to default
sys.stderr = stderr

# Reload the best scoring model.
model.load_weights(checkpoint_path)

In [ ]:
plt.plot(history["loss"], label = "Training loss")
plt.plot(history["val_loss"], label = "Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Crossentropy loss")
plt.show()

In [ ]:
plt.plot(history["f1"], label = "Training F1")
plt.plot(history["val_f1"], label = "Validation F1")
plt.xlabel("Epochs")
plt.ylabel("Macro F1 Score")
plt.show()

## Evaluation

In [ ]:
def predict_mask(X_target):
    if len(X_target.shape) == 3:
        # Add batch dimension
        X_target = tf.reshape(X_target, (1,) + X_target.shape)

    y_pred = tf.argmax(model(X_target), axis = -1)[0]

    return y_pred

In [ ]:
def to_img(y):
    """Convert class mask back to image for visualization."""
    out = np.zeros(y.shape[:2] + (3,))

    out[:, :, 0] = (y // 4)
    out[:, :, 1] = ((y % 4) // 2)
    out[:, :, 2] = (y % 2)

    return out

In [ ]:
def confusion_matrix_show(cm):
    class_labels = list(range(n_classes))
    sns.heatmap(cm,
                annot=True,
                fmt='g',
                cmap='Blues',
                xticklabels=class_labels,
                yticklabels=class_labels)

    plt.ylabel('Actual', fontsize=13)
    plt.title('Confusion Matrix', fontsize=17, pad=20)
    plt.gca().xaxis.set_label_position('top')
    plt.xlabel('Prediction', fontsize=13)
    plt.gca().xaxis.tick_top()

    plt.gca().figure.subplots_adjust(bottom=0.2)
    plt.gca().figure.text(0.5, 0.05, 'Prediction', ha='center', fontsize=13)
    plt.show()

In [ ]:
def Visualize_output(X_target, y_target):
    y_pred = predict_mask(X_target)
    img_pred = to_img(y_pred)
    img_real = to_img(y_target)

    y_target_flat = tf.reshape(y_target,-1)
    y_pred_flat = tf.reshape(y_pred, -1)

    cm = confusion_matrix(y_target_flat, y_pred_flat, labels = np.arange(n_classes))
    confusion_matrix_show(cm)

    fig, axes = plt.subplots(1, 3, figsize=(10, 5))

    # Plot real image
    axes[0].imshow(X_target, interpolation = 'nearest')
    axes[0].axis('off')
    axes[0].set_title("Real Image:")

    # Plot predicted mask
    axes[1].imshow(img_pred, interpolation = 'nearest')
    axes[1].axis('off')
    f1.reset_state()
    axes[1].set_title(f"Predicted Mask (F1: {f1(tf.one_hot(y_target_flat, n_classes), tf.one_hot(y_pred_flat, n_classes)):0.4f}):")

    # Plot target mask
    axes[2].imshow(img_real, interpolation = 'nearest')
    axes[2].axis('off')
    axes[2].set_title("Real Mask:")

    # Display the images
    plt.tight_layout()
    plt.show()

In [ ]:
random_indices = np.random.randint(0, n_val, 3)
for i in random_indices:
    Visualize_output(X_val[i], y_val[i])